# CogVideoX：让视频生成变得如此简单

**模型编码**：`cogvideox`

## 概述

CogVideoX 是由智谱 AI 开发的一款强大的视频生成大模型。它具备卓越的视频生成能力，只需输入文本或图片，就可以轻松完成视频制作。无论您是创作者、设计师，还是需要快速生成视频内容的用户，CogVideoX 都能满足您的需求，为您开启视频创作的新篇章。

## 模型特点

- **多模态输入**：支持文本和图像输入，灵活满足不同创作需求。
- **异步调用**：采用异步调用方式，合理利用资源，提高处理效率。
- **易于集成**：提供简单明了的 API 接口，方便开发者快速上手。

注意此处异步调用是指视频生成任务在服务器端可能是异步执行的（即你提交任务后，服务器在后台处理）：

- **同步服务器处理：** 任务处理和结果返回都是在一个请求中完成的，客户端发送请求后需要等待服务器处理完成，才能得到结果。

- **异步服务器处理：** 服务器在接收到任务请求后，立即返回一个任务ID，客户端可以通过这个ID去查询任务状态，实现非阻塞式的交互。




### 视频生成任务接口


#### 请求参数

| 参数名称   | 类型   | 是否必填 | 参数说明                                                                                                                                                                                                                                                                                                |
| ---------- | ------ | -------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| model      | String | 是       | 模型编码，取值为 `cogvideox`。                                                                                                                                                                                                                                                                           |
| prompt     | String | 可选     | 视频的文本描述，例如：“一只在海边奔跑的金毛犬，夕阳下显得格外活泼”。`image_url` 和 `prompt` 二选一或同时传入。                                                                                                                                                                                            |
| image_url  | String | 可选     | 提供用于生成内容的图像。支持通过 URL 或 Base64 编码传入图片。图片要求如下：<br>- 支持 `.png`、`.jpeg`、`.jpg` 格式<br>- 图片比例建议为 3:2<br>- 图片大小不超过 5MB<br>`image_url` 和 `prompt` 二选一或同时传入。                                                                                      |
| request_id | String | 否       | 用户自定义的请求编号，需保证唯一性，用于区分每次请求。如果不传入，平台会默认生成。                                                                                                                                                                                                                         |
| user_id    | String | 否       | 终端用户的唯一 ID，协助平台对终端用户的违规行为进行干预。ID 长度要求：最少 6 个字符，最多 128 个字符。                                                                                                                                                                                                     |

#### 请求参数详解

- **model**：指定要使用的模型，此处为 `cogvideox`。
- **prompt**：视频的文本描述，可以描述场景、人物、动作等元素，越详细越好。
- **image_url**：用于辅助生成视频的图像，提供更具体的视觉参考。可以使用图片的 URL 或 Base64 编码。
- **request_id**：用于标识请求的唯一编号，方便跟踪和查询。
- **user_id**：用于标识终端用户，协助平台进行内容安全管理。


**官网推荐的提示词公式：**(镜头语言 + 景别角度 + 光影) + 主体 (主体描述) + 主体运动 + 场景 (场景描述) + (氛围)

**例子：**

**1. 可爱猫咪玩耍:**

* **简单版：**一只橘猫在草地上玩耍
* **进阶版：**(跟随拍摄 + 近景 + 自然光) 一只毛茸茸的橘猫 (橘色条纹，蓝色眼睛)，(追逐着一只蝴蝶)，在绿油油的草地 (点缀着黄色的小花) 上玩耍，(氛围：活泼可爱)

但发现有时候要求很细反而大模型不知道怎么做，视频质量并不一定就比广泛的指定好看。

In [29]:
from zhipuai import ZhipuAI

# 请填写您自己的 APIKey
# client = ZhipuAI(api_key="YOUR_API_KEY")
client = ZhipuAI()

In [23]:
response = client.videos.generations(
    model="cogvideox",
    prompt="未来都市的夜景",
    request_id="unique_request_id_201"
)

print(response)

VideoObject(id='38501731306541122-9221302274440747947', model='cogvideox', video_result=None, task_status='PROCESSING', request_id='unique_request_id_201')


In [24]:
response_pro = client.videos.generations(
    model="cogvideox",
    prompt="(无人机航拍 + 鸟瞰视角 +霓虹灯光) 未来都市 (高耸入云的摩天大楼，空中飞行的汽车)，(车流穿梭，灯光闪烁)，(氛围：科技感，未来感)",
    request_id="unique_request_id_202"
)

print(response_pro)

VideoObject(id='38501731306541122-9221302549318884321', model='cogvideox', video_result=None, task_status='PROCESSING', request_id='unique_request_id_202')


#### 解析响应


#### 响应参数详解

- **request_id**：用户在客户端请求时提交的任务编号或者平台生成的任务编号。  用于匹配请求和响应，方便跟踪。
- **id**：任务的唯一标识符，在查询任务结果时需要使用。
- **task_status**：初始状态通常为 `PROCESSING`，表示任务正在处理中；`SUCCESS`，成功；`FAIL`，失败

In [25]:
response_id = response.id
print(response_id)

38501731306541122-9221302274440747947


In [26]:
response_pro_id = response_pro.id
print(response_pro_id)

38501731306541122-9221302549318884321


### 任务结果查询接口


#### 请求参数

| 参数名称 | 类型   | 是否必填 | 参数说明                    |
| -------- | ------ | -------- | --------------------------- |
| id       | String | 是       | 任务 id，即之前响应中的 `id` |

#### 响应参数

| 参数名称       | 类型   | 参数说明                                                                                                                                             |
| -------------- | ------ | ---------------------------------------------------------------------------------------------------------------------------------------------------- |
| model          | String | 模型名称。                                                                                                                                           |
| request_id     | String | 用户在客户端请求时提交的任务编号或者平台生成的任务编号。                                                                                             |
| task_status    | String | 处理状态，取值为：<br>- `PROCESSING`：处理中<br>- `SUCCESS`：成功<br>- `FAIL`：失败                                                                   |
| video_result   | List   | 视频生成结果列表，仅在 `task_status` 为 `SUCCESS` 时返回。                                                                                            |
| ├─ url         | String | 视频的下载链接。                                                                                                                                     |
| ├─ cover_image_url | String | 视频封面的图片链接。                                                                                                                        |
| id             | String | 智谱 AI 开放平台生成的任务 id。                                                                                                                      |


In [27]:
result = client.videos.retrieve_videos_result(
    id=response_id,
)

print(result)

VideoObject(id=None, model='cogvideox', video_result=[VideoResult(url='https://aigc-files.bigmodel.cn/api/cogvideo/55ac1a9e-a7f2-11ef-a354-628f45da38f5_0.mp4', cover_image_url='https://aigc-files.bigmodel.cn/api/cogvideo/55ac1a9e-a7f2-11ef-a354-628f45da38f5_cover_0.jpeg')], task_status='SUCCESS', request_id='unique_request_id_201')


In [28]:
result_pro = client.videos.retrieve_videos_result(
    id=response_pro_id,
)

print(result_pro)

VideoObject(id=None, model='cogvideox', video_result=[VideoResult(url='https://aigc-files.bigmodel.cn/api/cogvideo/59ca3b06-a7f2-11ef-a354-628f45da38f5_0.mp4', cover_image_url='https://aigc-files.bigmodel.cn/api/cogvideo/59ca3b06-a7f2-11ef-a354-628f45da38f5_cover_0.jpeg')], task_status='SUCCESS', request_id='unique_request_id_202')


#### 响应示例

#### 响应参数详解

- **task_status**：指示当前任务的状态，需要根据此状态判断下一步操作。
- **video_result**：当任务成功时，包含生成的视频及封面链接。

任务处理中：

In [ ]:
{
    "model": "cogvideox",
    "request_id": "unique_request_id_001",
    "task_status": "PROCESSING",
    "id": "1234567890123456789"
}

任务成功：

In [ ]:
{
    "model": "cogvideox",
    "request_id": "unique_request_id_001",
    "task_status": "SUCCESS",
    "video_result": [
        {
            "cover_image_url": "https://example.com/video_cover.png",
            "url": "https://example.com/generated_video.mp4"
        }
    ],
    "id": "1234567890123456789"
}

## 完整使用流程

### 第一步：提交视频生成任务

In [8]:
# 提交任务
response = client.videos.generations(
    model="cogvideox",
    prompt="一个孩子在草地上放风筝，天空晴朗",
    request_id="unique_request_id_002"
)

# 获取任务 id
task_id = response.id
print(f"任务已提交，任务 ID：{task_id}")

任务已提交，任务 ID：38501731306541122-9221298735386559931


### 第二步：定期查询任务状态

In [9]:
import time

while True:
    # 查询任务状态
    result = client.videos.retrieve_videos_result(id=task_id)
    status = result.task_status
    print(f"当前任务状态：{status}")

    if status == "SUCCESS":
        # 任务成功，获取结果
        video_url = result.video_result[0].url
        cover_image_url = result.video_result[0].cover_image_url
        print(f"视频生成成功！\n视频地址：{video_url}\n封面地址：{cover_image_url}")
        break
    elif status == "FAIL":
        print("视频生成失败，请检查输入参数或联系支持。")
        break
    else:
        # 等待一段时间后再次查询
        time.sleep(5)

当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：SUCCESS
视频生成成功！
视频地址：https://aigc-files.bigmodel.cn/api/cogvideo/16da2836-a7ea-11ef-bd78-9ee7942420ea_0.mp4
封面地址：https://aigc-files.bigmodel.cn/api/cogvideo/16da2836-a7ea-11ef-bd78-9ee7942420ea_cover_0.jpeg


同时传入 `prompt` 和 `image_url` 
`prompt` 和 `image_url` 可以同时传入，以提供更丰富的生成条件，模型将综合考虑文本和图像信息生成视频。
有两种主要图片来源的读取与处理方法：

1. 从本地文件系统读取图像，要将本地图像传递，我们需要将图像内容转换为 base64 编码的字符串。
2. 从互联网 URL 获取在线图像，注意ulr的后缀应该为图片的格式，例如"https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png"

**图像数据处理秘笈：base64 编码的奥秘**

为何要将图像数据转换成base64传递

- **二进制数据**：图像文件以二进制形式存储像素信息。每个像素通常由红、绿、蓝（RGB）三个颜色通道的值组成。
- **Base64 编码**：将二进制数据转换为由 64 个特定字符组成的字符串。这些字符包括大写 A-Z、小写 a-z、数字 0-9、加号 + 和斜杠 /。Base64 编码确保数据可以在文本处理系统中安全传输。



In [ ]:
import base64
from zhipuai import ZhipuAI

video_path = "/Users/YourCompluter/xxxx.jpg"
with open(video_path, 'rb') as video_file:
    image_base = base64.b64encode(video_file.read()).decode('utf-8')

client = ZhipuAI(api_key="YOUR API KEY") # 填写您自己的APIKey
response = client.chat.completions.create(
    model="cogvideox",  # 填写需要调用的模型名称
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "image_url",
            "video_url": {
                "url" : image_base
            }
          },
          {
            "type": "text",
            "text": "请仔细描述这个视频"
          }
        ]
      }
    ]
)
print(response.choices[0].message)

In [11]:
# 提交任务
response = client.videos.generations(
    model="cogvideox",
    prompt="星空的线条在动",
    image_url="https://cdn.bigmodel.cn/static/platform/images/usage-guide/cogvideo/1.png",
    request_id="unique_request_id_003"
)

# 获取任务 id
task_id = response.id
print(f"任务已提交，任务 ID：{task_id}")

任务已提交，任务 ID：38501731306541122-9221298632307196664


In [12]:
import time

while True:
    # 查询任务状态
    result = client.videos.retrieve_videos_result(id=task_id)
    status = result.task_status
    print(f"当前任务状态：{status}")

    if status == "SUCCESS":
        # 任务成功，获取结果
        video_url = result.video_result[0].url
        cover_image_url = result.video_result[0].cover_image_url
        print(f"视频生成成功！\n视频地址：{video_url}\n封面地址：{cover_image_url}")
        break
    elif status == "FAIL":
        print("视频生成失败，请检查输入参数或联系支持。")
        break
    else:
        # 等待一段时间后再次查询
        time.sleep(5)

当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：PROCESSING
当前任务状态：SUCCESS
视频生成成功！
视频地址：https://aigc-files.bigmodel.cn/api/cogvideo/19962d0c-a7ec-11ef-96bb-e63bd27113c0_0.mp4
封面地址：https://aigc-files.bigmodel.cn/api/cogvideo/19962d0c-a7ec-11ef-96bb-e63bd27113c0_cover_0.jpeg


### 第三步：下载和使用视频

在任务成功后，您可以使用返回的 `url` 和 `cover_image_url` 下载视频和封面图片，进行后续处理或展示。

## 使用建议

- **详细描述**：在 `prompt` 中提供详细、具体的描述，有助于生成更符合预期的视频内容。
- **合理安排查询频率**：建议每隔 5 秒至 10 秒查询一次任务状态，避免过于频繁的请求。
- **注意内容合规**：请确保输入的文本和图片不包含敏感或违规内容。

## 常见问题解答

### 1. 任务一直处于 `PROCESSING` 状态怎么办？

视频生成可能需要一定的处理时间，特别是当服务器负载较高时。请耐心等待，并定期查询任务状态。如果长时间（如超过 30 分钟）仍未完成，建议联系平台支持。

### 2. 任务失败的常见原因有哪些？

- **输入参数不合法**：如图片格式不支持、图片大小超过限制、文本描述包含敏感词汇等。
- **服务器异常**：可能由于服务器故障导致任务失败，您可以稍后重试。

### 3. 如何确保 `request_id` 的唯一性？

`request_id` 可以使用 UUID、时间戳加随机数等方式生成，确保在您的应用中每次请求都使用不同的 `request_id`。
